In [1]:
import pandas as pd 
import plotly.graph_objects as go

C:\Users\pellan\AppData\Local\anaconda3\envs\rslab\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\pellan\AppData\Local\Temp\ipykernel_13932\1966795625.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_excel(r'footprint_sectors.xlsx')

In [3]:
data['Exiobase_sectors'] = data['Exiobase_sectors'].apply(lambda x: x[:10] + '...' if len(x) > 15 else x)

In [4]:
data

,Exiobase_sectors,IEA_sectors,SECTEN_sectors,Embodied footprint
0,Paddy rice,Agriculture & forestry,Agriculture,0.174735
1,Wheat,Agriculture & forestry,Agriculture,0.108195
2,Cereal gra...,Agriculture & forestry,Agriculture,0.132983
3,Vegetables...,Agriculture & forestry,Agriculture,0.047702
4,Oil seeds,Agriculture & forestry,Agriculture,0.100685
...,...,...,...,...
195,Membership...,"Public adm;education,health",Services,0.002319
196,Recreation...,"Public adm;education,health",Services,0.035365
197,Other serv...,"Public adm;education,health",Services,0.011682
198,Private ho...,"Public adm;education,health",Services,0.001200


In [9]:
# Correcting the approach based on the original column names provided
data.columns = ['Exiobase_sectors', 'IEA_sectors', 'SECTEN_sectors', 'Embodied_footprint']

# Apply unique naming for IEA and SECTEN sectors to avoid loops and make them distinguishable
data['IEA_sectors'] = data['IEA_sectors'].apply(lambda x: f"{x} ")
data['SECTEN_sectors'] = data['SECTEN_sectors'].apply(lambda x: f"{x}")

# Define nodes again with the updated naming
nodes = pd.concat([
    data['Exiobase_sectors'], 
    data['IEA_sectors'], 
    data['SECTEN_sectors']
]).unique()

# Update the mapping for these nodes with the new unique labels
node_ids = {node: i for i, node in enumerate(nodes)}

# Initialize lists again for sources, targets, and values with corrected data
sources = []
targets = []
values = []

# Mapping Exiobase_sectors to IEA_sectors and IEA_sectors to SECTEN_sectors with unique mappings
for _, row in data.iterrows():
    sources.append(node_ids[row['Exiobase_sectors']])
    targets.append(node_ids[row['IEA_sectors']])
    values.append(row['Embodied_footprint'])

iea_to_secten = data[['IEA_sectors', 'SECTEN_sectors', 'Embodied_footprint']].drop_duplicates()
for _, row in iea_to_secten.iterrows():
    sources.append(node_ids[row['IEA_sectors']])
    targets.append(node_ids[row['SECTEN_sectors']])
    values.append(row['Embodied_footprint'])

# Create the updated Sankey diagram
fig_updated_corrected = go.Figure(data=[go.Sankey(
    node=dict(
        pad=10,
        thickness=20,
        line=dict(color="black", width=1.5),
        label=list(nodes)
    ),
    link=dict(
        source=sources,  # Source nodes
        target=targets,  # Target nodes
        value=values    # Flow magnitudes
        #color="#adb5bd", 
    ),
    orientation='v'

    )])

# Adjust the overall font size of the diagram, including node labels
fig_updated_corrected.update_layout(
    title_text="", 
    font=dict(
        size=35,  # Adjust the font size as needed # 23
        color="black",  # Set the font color
        family="Arial Black",  # Specify a font family that supports bold style
    ),
    width=2000, #1800
    height=2400 #2200
)

# Save the updated diagram as an HTML file for download
output_file_path_corrected = 'sankey.html'
fig_updated_corrected.write_html(output_file_path_corrected)

output_file_path_corrected


'sankey.html'